<a href="https://colab.research.google.com/github/shruthimohan03/video-summarizer/blob/main/Final_Entire_Text_Summarization_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Input Acquisition

In [ ]:
 inputs = [["https://www.youtube.com/watch?v=tA42nHmmEKw","nptel_python_transcript_preprocessed.txt","nptel_python_summary"],
          ["https://youtu.be/Ero4yY_ttJE?si=S-6ADdxl4S7EXDkc","federated_learning_transcript.txt","federated_learning_summary.txt"],
          ["https://youtu.be/xRmBEnI55es?si=jftY1j9b8C9EvqJa","cs_transcript.txt","cs_summary.txt"],
          ["https://www.youtube.com/watch?v=VyzKDc2GyW4","normalization_transcript.txt","normalization_summary.txt"],
          ["https://www.youtube.com/watch?v=t45S_MwAcOw",'transformer_transcript.txt','transformer_summary.txt']]
# youtube_url , file_transcript , file_summary

### 2. Audio Processing Pipeline

#### 2.1. Audio Extraction

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
pip install yt-dlp pydub SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 8.8 MB/s eta 0:00:00


In [ ]:
import os
from yt_dlp import YoutubeDL

def download_audio_from_youtube(youtube_url):
    options = {
        'format': 'bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',  # Use video ID to avoid filename issues
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }

    with YoutubeDL(options) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        video_id = info['id']
        wav_file = f"{video_id}.wav"

        if os.path.exists(wav_file):
            print(f"Downloaded and converted audio to WAV: {wav_file}")
            return wav_file
        else:
            raise FileNotFoundError(f"WAV file {wav_file} not found.")

#### 2.2 Audio-to-Text Conversion

In [ ]:
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 26.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import whisper
def transcribe_audio(audio_file_path):
    # Load the Whisper model
    model = whisper.load_model("small", device="cuda")

    print('Transcribing...')
    # Transcribe the audio
    result = model.transcribe(audio_file_path)
    print('Transcription is ready')
    return result["text"]

### 3. Text Processing Pipeline

#### 3.1 Punctuation Correction and Preprocessing

In [ ]:
pip install language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import language_tool_python

def punctuate_text(text):
    tool = language_tool_python.LanguageTool('en-US')

    # Only correct punctuation-related errors
    punctuation_matches = [
        match for match in tool.check(text)
        if 'PUNCTUATION' in match.ruleIssueType.upper()
    ]
    # Apply the corrections
    corrected_text = language_tool_python.utils.correct(text, punctuation_matches)
    return corrected_text

#### EVALUATION OF THE TRANSCRIPT: USING WORD ERROR RATE

In [ ]:
# pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.9 MB/s eta 0:00:00


In [ ]:
'''def import_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

reference_file_path = "/kaggle/input/transcripts/reference_nptel_python_transcript.txt"
reference = import_txt_file(reference_file_path)'''

'\nreference_file_path = "/kaggle/input/transcripts/reference_nptel_python_transcript.txt"\nreference = import_txt_file(reference_file_path)'

In [ ]:
'''import re
import string

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove newlines and extra spaces
    text = re.sub(r'\s+', ' ', text)

    # Strip leading and trailing spaces
    text = text.strip()

    return text

cleaned_reference = preprocess_text(reference)
cleaned_hypothesis = preprocess_text(corrected)'''

'cleaned_reference = preprocess_text(reference)\ncleaned_hypothesis = preprocess_text(corrected)'

In [ ]:
'''from jiwer import wer

error_rate = wer(cleaned_reference, cleaned_hypothesis)
print("Word Error Rate:", error_rate)'''

#### 3.2. Extractive Text Summarization

a. Sentence Segmentation

In [ ]:
def sentence_segmentation(text):
  from nltk.tokenize import sent_tokenize
  sentences = sent_tokenize(text)
  sentences = [s for s in sentences if len(s.split()) > 3]
  return sentences

b. Sentence Embedding with SBERT

In [ ]:
def embedding(sentences):
  from sentence_transformers import SentenceTransformer
  model = SentenceTransformer('all-MiniLM-L6-v2')
  sentence_embeddings = model.encode(sentences, convert_to_tensor=False)
  return sentence_embeddings

c. Optimal Cluster Estimation using Bayesian Information Criterion (BIC)

##### Smart Thresholding for range of clusters depending on the number of sentences present in the transcript

In [ ]:
import math

def get_cluster_range_percent(N, min_ratio=0.05, max_ratio=0.2):
    min_c = max(3, math.ceil(N * min_ratio))
    max_c = min(N - 1, math.ceil(N * max_ratio))
    return (min_c, max_c)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Range of clusters to try
def get_cluster_range(N):
  clusters=get_cluster_range_percent(N)
  n_components = np.arange(clusters[0],clusters[1]+1)
  return n_components

In [ ]:
def get_BIC(n_components,sentence_embeddings):
  from sklearn.mixture import GaussianMixture
  bics = []
  for n in n_components:
      gmm = GaussianMixture(n_components=n, covariance_type='full', random_state=42)
      gmm.fit(sentence_embeddings)
      bics.append(gmm.bic(sentence_embeddings))
  return bics

##### Plotting BIC values

In [ ]:
def plot_BIC(n_components,bics):
  plt.plot(n_components, bics, marker='o')
  plt.xlabel('Number of clusters')
  plt.ylabel('BIC Score')
  plt.title('BIC to choose optimal number of clusters')
  plt.show()

##### Finding the optimal number of clusters

In [ ]:
def find_optimal_clusters(bics):
  optimal_clusters = n_components[np.argmin(bics)]
  print("Optimal number of clusters:", optimal_clusters)
  return optimal_clusters

d. Sentence Clustering using Gaussian Mixture Models (GMM) (the optimal no of clusters is used)

In [ ]:
def best_gmm_model(optimal_clusters,sentence_embeddings):
  from sklearn.mixture import GaussianMixture
  best_gmm = GaussianMixture(n_components=optimal_clusters, covariance_type='full', random_state=42)
  best_gmm.fit(sentence_embeddings)

  # Predict cluster labels
  labels = best_gmm.predict(sentence_embeddings)
  return labels,best_gmm

e. Summary Sentence Selection

In [ ]:
def summary_sentences(labels, optimal_clusters, sentences, sentence_embeddings, best_gmm):
  import numpy as np
  from sklearn.metrics.pairwise import cosine_similarity
  from math import ceil

  summary_sentences = []
  seen_indices = set()

  # You can tune this ratio
  summary_ratio = 0.2
  min_sentences_per_cluster = 1
  max_sentences_per_cluster = 3

  for cluster_idx in range(optimal_clusters):
      cluster_indices = np.where(labels == cluster_idx)[0]
      cluster_size = len(cluster_indices)

      # Skip clusters with only one sentence (likely noise)
      if cluster_size <= 1:
          continue

      cluster_embeddings = np.array([sentence_embeddings[i] for i in cluster_indices])
      cluster_sentences = [sentences[i] for i in cluster_indices]

      # Get centroid of the cluster
      centroid = best_gmm.means_[cluster_idx].reshape(1, -1)

      # Rank all cluster sentences by similarity to centroid
      similarities = cosine_similarity(centroid, cluster_embeddings)[0]
      sorted_idx = np.argsort(similarities)[::-1]  # Descending order

      # Determine how many sentences to extract from this cluster
      num_to_select = min(
          max(min_sentences_per_cluster, ceil(summary_ratio * cluster_size)),
          max_sentences_per_cluster
      )

      selected_indices = sorted_idx[:num_to_select]

      for idx in selected_indices:
          global_idx = cluster_indices[idx]

          if global_idx not in seen_indices:
              summary_sentences.append((global_idx, sentences[global_idx]))
              seen_indices.add(global_idx)

  # Sort by original sentence order for coherence
  summary_sentences = sorted(summary_sentences, key=lambda x: x[0])

  # Final summary
  summary = " ".join([s[1] for s in summary_sentences])

  # Print summary
  for _, sent in summary_sentences:
      print(sent)
  return summary

### RUN THE ENTIRE AUDIO + TEXT PIPELINE

In [ ]:
for item in inputs:
  # 2.1) Audio Extraction
  wav_file = download_audio_from_youtube(item[0])

  # 2.2) Audio-to-Text Conversion
  transcription = transcribe_audio(wav_file)
  print("\nTranscription:\n")
  print(transcription)

  # 3.1) Punctuation Correction and Preprocessing
  corrected = punctuate_text(transcription)
  print("Corrected:", corrected)

  # saving the preprocessed transcript
  with open(item[1], "w") as file:
    file.write(corrected)
  print(f"Transcription has been saved to {item[1]}")

  ## EXTRACTIVE SUMMARIZATION:
  # a. Sentence Segmentation
  sentences=sentence_segmentation(corrected)

  # b. Sentence Embedding with SBERT
  sentence_embeddings=embedding(sentences)

  # c. Optimal Cluster Estimation using Bayesian Information Criterion (BIC)
  N = len(sentences)
  n_components=get_cluster_range(N)
  bics=get_BIC(n_components,sentence_embeddings)
  ## plot
  #plot_BIC(n_components,bics)
  optimal_clusters=find_optimal_clusters(bics)

  # d. Sentence Clustering using Gaussian Mixture Models (GMM) (the optimal no of clusters is used)
  labels,best_gmm=best_gmm_model(optimal_clusters,sentence_embeddings)
  summary=summary_sentences(labels, optimal_clusters, sentences, sentence_embeddings, best_gmm)

  # saving the summary
  with open(item[2],'w') as file:
    file.write(summary)
  print('Summary file saved')

[youtube] Extracting URL: https://www.youtube.com/watch?v=tA42nHmmEKw
[youtube] tA42nHmmEKw: Downloading webpage
[youtube] tA42nHmmEKw: Downloading tv client config
[youtube] tA42nHmmEKw: Downloading player 9599b765-main
[youtube] tA42nHmmEKw: Downloading tv player API JSON
[youtube] tA42nHmmEKw: Downloading ios player API JSON
[youtube] tA42nHmmEKw: Downloading m3u8 information
[info] tA42nHmmEKw: Downloading 1 format(s): 251-6
[download] Destination: tA42nHmmEKw.webm
[download] 100% of   13.18MiB in 00:00:01 at 11.38MiB/s  
[ExtractAudio] Destination: tA42nHmmEKw.wav
Deleting original file tA42nHmmEKw.webm (pass -k to keep)
Downloaded and converted audio to WAV: tA42nHmmEKw.wav
Transcribing...
Transcription is ready

Transcription:

 Welcome to this course on Python for Data Science. This is a 4-week course where we are going to teach you some very basic programming aspects in Python. And since this is a course that is geared towards data science towards end of the course based on wh